In [1]:
import pandas as pd
import argparse
import textwrap
from chembl_webresource_client.new_client import new_client
from rdkit.Chem import AllChem
import os
from pandas import json_normalize
import requests
from json import JSONDecodeError
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'NP_Score'))
import npscorer
from argparse import Namespace

In [2]:
# Define the output folder path
output_folder = 'C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/'  # Replace 'chembl_custom_folder' with your desired folder name

# Define the arguments
args_dict = {
    "target_id": "CHEMBL3051",  # Replace with your default target ID
    "NPlike_score": -1  # Replace with your default NPlike_score
}

# Convert the arguments to Namespace object
args = Namespace(**args_dict)

In [3]:
""" Functions """

# Function used to clean the data downloaded from ChEMBL
def clean_DB(df_in, NP_model, NP_cutoff):
    '''Function to clean a ChEMBL DB'''
    
    df=df_in.copy()
    isomeric_smiles = []
    np_scores = []
    inchikey = []

    # Drop rows without Smiles or activity value
    #df.dropna(subset=['smiles', 'standard_value'], inplace=True)

    # Drop row with a Data Validity Comment "Outside typical range"
    #df.drop(df[df['data_validity_comment'] == "Outside typical range"].index, inplace=True)
    #df.drop(['data_validity_comment', 'relation', 'units', 'value', 'activity_comment', 'type'],axis =1, inplace=True)
    
    # Drop rows with an invalid smiles, replace smiles with canonical smiles and add a columns with smiles without stereo-isomers
    mols = []
    
    for i, row in df.iterrows():
        mol = AllChem.MolFromSmiles(row["canonical_smiles"])
        if mol is not None:
            iso_smiles = AllChem.MolToSmiles(mol, isomericSmiles=True)
            isomeric_smiles.append(iso_smiles)
            inchikey.append(AllChem.MolToInchiKey(mol))
            np_scores.append(npscorer.scoreMol(mol, NP_model))
            row["canonical_smiles"] = AllChem.MolToSmiles(mol)
        else:
            df.drop(i, inplace=True)

    # l=[i for i in range(len(mols)) if mols[i] == None]
    # df.drop(df.index[l], inplace=True)

    # for _, row in df.iterrows():
    #     mol = AllChem.MolFromSmiles(row["canonical_smiles"])
    #     iso_smiles = AllChem.MolToSmiles(mol, isomericSmiles=True)
    #     isomeric_smiles.append(iso_smiles)
    #     inchikey.append(AllChem.MolToInchiKey(mol))

    df['isomeric_smiles'] = isomeric_smiles
    df['inchikey'] = inchikey
    df['np_score'] = np_scores
    # Convert 'inchikey' column to string type
    df['inchikey'] = df['inchikey'].astype(str)

    # Now you can safely use the .str accessor on the 'inchikey' column
    df['short_inchikey'] = df['inchikey'].str[:14]
    df["document_journal"].replace({None: "Unknown journal"}, inplace=True)
    df = df[(df['np_score'] > NP_cutoff) | (df['document_journal'] == 'J Nat Prod')]
    
    return df


def get_all_ik(url):
    query = '''
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            SELECT ?ik ?wd ?isomeric_smiles
            WHERE{
                ?wd wdt:P235 ?ik .
                optional { ?wd wdt:P2017 ?isomeric_smiles } 
            }
            '''
    try:
        r = requests.get(url, params={'format': 'json', 'query': query})
        r.raise_for_status()  # Raise an exception for HTTP errors
        data = r.json()
        results = pd.DataFrame.from_dict(data).get('results', {}).get('bindings', [])
        df = json_normalize(results)
        df.rename(columns={'wd.value':'wikidata_id', 'ik.value':'inchikey', 'isomeric_smiles.value': 'isomeric_smiles'}, inplace=True)
        return df[['wikidata_id', 'inchikey', 'isomeric_smiles']]
    except (JSONDecodeError, requests.RequestException) as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of errors


In [11]:
# Download selected activities
activities = new_client.activity
activities = activities.filter(target_chembl_id = args.target_id)
    
res = activities.filter(standard_value__isnull=False) # Keep only compounds with an activity value
NP_model = npscorer.readNPModel()

#res = activities.filter(standard_type__iexact = 'IC50')
res = res.only(['activity_comment', 'molecule_chembl_id', 'canonical_smiles', 'standard_relation', 'target_chembl_id',
            'standard_type', 'target_pref_name', 'standard_units', 'standard_value', 'data_validity_comment', 'document_journal',
            'assay_chembl_id', 'document_chembl_id'
             ])

print('Fetching results from ChEMBL: this step can be long. Have a coffee ;)')
res_df = pd.DataFrame.from_dict(res)
print('Fetching results from ChEMBL: Done!')
#res_df.head()
df_clean = clean_DB(res_df, NP_model, int(args.NPlike_score))
df_clean.head()

reading NP model ...
model in


Fetching results from ChEMBL: this step can be long. Have a coffee ;)
Fetching results from ChEMBL: Done!


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\3017324297.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["document_journal"].replace({None: "Unknown journal"}, inplace=True)


,activity_comment,assay_chembl_id,canonical_smiles,data_validity_comment,document_chembl_id,document_journal,molecule_chembl_id,relation,standard_relation,standard_type,...,standard_value,target_chembl_id,target_pref_name,type,units,value,isomeric_smiles,inchikey,np_score,short_inchikey
0,None,CHEMBL696856,CN(CCC[C@H](N)C(=O)O)C(=N)N,None,CHEMBL1127683,J Med Chem,CHEMBL99636,=,=,Inhibition,...,89.0,CHEMBL3051,"Nitric oxide synthase, inducible",Inhibition,%,89.0,CN(CCC[C@H](N)C(=O)O)C(=N)N,XKCWNEVAXQCMGP-YFKPBYRVSA-N,0.785323,XKCWNEVAXQCMGP
1,None,CHEMBL872049,CN(CCC[C@H](N)C(=O)O)C(=N)N,None,CHEMBL1127683,J Med Chem,CHEMBL99636,=,=,Inhibition,...,33.0,CHEMBL3051,"Nitric oxide synthase, inducible",Inhibition,%,33.0,CN(CCC[C@H](N)C(=O)O)C(=N)N,XKCWNEVAXQCMGP-YFKPBYRVSA-N,0.785323,XKCWNEVAXQCMGP
2,None,CHEMBL696856,N/C(S)=N/CCC[C@@H](N)C(=O)O,None,CHEMBL1127683,J Med Chem,CHEMBL168461,<,<,Inhibition,...,1.0,CHEMBL3051,"Nitric oxide synthase, inducible",Inhibition,%,1.0,N/C(S)=N/CCC[C@@H](N)C(=O)O,BKGWACHYAMTLAF-SCSAIBSYSA-N,0.994133,BKGWACHYAMTLAF
3,None,CHEMBL872049,CSC(=N)NCCC[C@@H](N)C(=O)O,None,CHEMBL1127683,J Med Chem,CHEMBL1161525,<,<,Inhibition,...,1.0,CHEMBL3051,"Nitric oxide synthase, inducible",Inhibition,%,1.0,CSC(=N)NCCC[C@@H](N)C(=O)O,NGVMVBQRKZPFLB-RXMQYKEDSA-N,0.492889,NGVMVBQRKZPFLB
4,None,CHEMBL696856,N/C(S)=N/CCCC[C@H](N)C(=O)O,None,CHEMBL1127683,J Med Chem,CHEMBL171397,=,=,Inhibition,...,75.0,CHEMBL3051,"Nitric oxide synthase, inducible",Inhibition,%,75.0,N/C(S)=N/CCCC[C@H](N)C(=O)O,NTDQCCFHWYKFAW-YFKPBYRVSA-N,0.929818,NTDQCCFHWYKFAW


In [10]:
#recover information from Wikidata 

wd_url = 'https://query.wikidata.org/sparql'
wd_all = get_all_ik(wd_url)
#wd_all.head()
wd_filtred = wd_all[wd_all['inchikey'].isin(list(df_clean.inchikey))]

#merge with the clean df
df_total = df_clean.merge(wd_filtred[['inchikey', 'wikidata_id']], on='inchikey', how='outer')
df_total['wikidata_id'] = df_total['wikidata_id'] .fillna('no_wikidata_match')
df_total.head()

NameError: name 'df_clean' is not defined

In [ ]:
# Construct the file path
filename = f"{args.target_id}_np_like_min_{args.NPlike_score}.csv"
path_to_file = os.path.join(output_folder, filename)

# Save the DataFrame to CSV
df_total.to_csv(path_to_file, index=False)

# Print a message indicating where the file was saved
print(f"Finished. Results are in: {path_to_file}")

Finished. Results are in: C:/Users/quirosgu/Documents/GitHub/Yggdrasil/CHEMBL2111350_np_like_min_-1.csv


MERGE INFORMATION if there were more than one CHEMBLcard


In [12]:
# Define the output folder containing multiple CSV files
output_folder = 'C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/'

# Initialize an empty DataFrame to store concatenated data
all_data = pd.DataFrame()

# Iterate over each CSV file in the output folder
for file in os.listdir(output_folder):
    if file.endswith('.csv'):
        # Read the CSV file into a DataFrame
        file_path = os.path.join(output_folder, file)
        df = pd.read_csv(file_path)
        
        # Concatenate the DataFrame with the existing data
        all_data = pd.concat([all_data, df], ignore_index=True)

# Group the DataFrame by 'inchikey' and aggregate the values in other columns
grouped_data = all_data.groupby('inchikey').agg('first').reset_index()

# Save the aggregated DataFrame to a new CSV file
output_csv_path = os.path.join(output_folder, 'merged_data.csv')
grouped_data.to_csv(output_csv_path, index=False)

print(f"Merged data saved to: {output_csv_path}")


Merged data saved to: C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/merged_data.csv


In [36]:
# Load merged_data.csv containing active compounds
merged_data = pd.read_csv('C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/merged_data.csv')

# Initialize an empty DataFrame to store taxonomical data
taxonomical_data = pd.DataFrame()

# Specify the folder containing TSV files with reported compounds for each species
tsv_folder = 'C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data/'

# Columns to select from the merged_data DataFrame
merged_columns = ['inchikey', 'short_inchikey', 'molecule_chembl_id', 'target_pref_name', 'units', 'value', 'wikidata_id']  # Add other columns from merged_data if needed

# Columns to select from the TSV files
tsv_columns = ['structure_inchikey', 'short_inchikey','structure_wikidata', 'structure_inchi', 
               'structure_molecular_formula', 'structure_exact_mass', 'structure_smiles_2D',
               'structure_nameIupac', 'structure_nameTraditional', 
               'structure_taxonomy_npclassifier_01pathway', 'structure_taxonomy_npclassifier_02superclass',
               'structure_taxonomy_npclassifier_03class', 'organism_wikidata', 
               'organism_taxonomy_ottid', 'organism_taxonomy_06family', 
               'organism_taxonomy_08genus', 'organism_taxonomy_09species', 
               'reference_wikidata', 'reference_doi', 'chemical_superclass']

# Iterate through each TSV file in the folder
for filename in os.listdir(tsv_folder):
    if filename.endswith('.tsv'):
        # Read the TSV file into a DataFrame
        tsv_filepath = os.path.join(tsv_folder, filename)
        species_data = pd.read_csv(tsv_filepath, sep='\t')

        # Filter the data to include only active compounds
        active_compounds_data = species_data[species_data['structure_inchikey'].isin(merged_data['inchikey'])]
        
        # Select desired columns from both DataFrames
        merged_selected = merged_data[merged_columns]
        tsv_selected = active_compounds_data[tsv_columns]
        
        # Concatenate the selected columns
        selected_data = pd.concat([merged_selected, tsv_selected], axis=1)
        
        # Append the selected data to the taxonomical_data DataFrame
        taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)

# Filter taxonomical_data to remove rows with empty 'structure_inchikey'
taxonomical_data = taxonomical_data.dropna(subset=['structure_inchikey'])

# Save the taxonomical data to a CSV file
taxonomical_data.to_csv('C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/species_with_active_compounds.csv', index=False)

print("Taxonomical data saved to species_taxonomical_data.csv")


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\22993683.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\22993683.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\22993683.py:41: FutureWarning: The behavior of Data

Taxonomical data saved to species_taxonomical_data.csv


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\22993683.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)


In [40]:
# Load merged_data.csv containing active compounds
merged_data = pd.read_csv('C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/merged_data.csv')

# Initialize an empty DataFrame to store taxonomical data
taxonomical_data = pd.DataFrame()

# Specify the folder containing TSV files with reported compounds for each species
tsv_folder = 'C:/Users/quirosgu/Desktop/Inmuno/output_data/species_data/'

# Columns to select from the merged_data DataFrame
merged_columns = ['inchikey', 'molecule_chembl_id', 'target_pref_name', 'units', 'value', 'wikidata_id']

# Columns to select from the TSV files
tsv_columns = ['structure_inchikey', 'structure_wikidata', 'structure_inchi', 
               'structure_molecular_formula', 'structure_exact_mass', 'structure_smiles_2D',
               'structure_nameIupac', 'structure_nameTraditional', 
               'structure_taxonomy_npclassifier_01pathway', 'structure_taxonomy_npclassifier_02superclass',
               'structure_taxonomy_npclassifier_03class', 'organism_wikidata', 
               'organism_taxonomy_ottid', 'organism_taxonomy_06family', 
               'organism_taxonomy_08genus', 'organism_taxonomy_09species', 
               'reference_wikidata', 'reference_doi', 'chemical_superclass']

# Iterate through each TSV file in the folder
for filename in os.listdir(tsv_folder):
    if filename.endswith('.tsv'):
        # Read the TSV file into a DataFrame
        tsv_filepath = os.path.join(tsv_folder, filename)
        species_data = pd.read_csv(tsv_filepath, sep='\t')

        # Merge the merged_data with species_data based on 'inchikey'
        matched_data = pd.merge(merged_data, species_data, left_on='inchikey', right_on='structure_inchikey', how='inner')
        
        # Select desired columns
        selected_data = matched_data[merged_columns + tsv_columns]
        
        # Append the selected data to the taxonomical_data DataFrame
        taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)

# Save the taxonomical data to a CSV file
taxonomical_data.to_csv('C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/species_with_active_compounds.csv', index=False)

print("Taxonomical data saved to species_taxonomical_data.csv")


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of D

Taxonomical data saved to species_taxonomical_data.csv


C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  taxonomical_data = pd.concat([taxonomical_data, selected_data], ignore_index=True)
C:\Users\quirosgu\AppData\Local\Temp\ipykernel_15432\489169755.py:37: FutureWarning: The behavior of D

In [6]:
# Save the taxonomical data to a CSV file
df = pd.read_csv('C:/Users/quirosgu/Documents/GitHub/Yggdrasil/chembl/species_with_active_compounds.csv',sep=',')
df.columns

Index(['inchikey', 'molecule_chembl_id', 'target_pref_name', 'units', 'value',
       'wikidata_id', 'structure_inchikey', 'structure_wikidata',
       'structure_inchi', 'structure_molecular_formula',
       'structure_exact_mass', 'structure_smiles_2D', 'structure_nameIupac',
       'structure_nameTraditional',
       'structure_taxonomy_npclassifier_01pathway',
       'structure_taxonomy_npclassifier_02superclass',
       'structure_taxonomy_npclassifier_03class', 'organism_wikidata',
       'organism_taxonomy_ottid', 'organism_taxonomy_06family',
       'organism_taxonomy_08genus', 'organism_taxonomy_09species',
       'reference_wikidata', 'reference_doi', 'chemical_superclass'],
      dtype='object')

In [10]:
# Count occurrences of each compound in each taxon
compound_counts = df.groupby(['structure_nameTraditional', 'organism_taxonomy_06family', 'organism_taxonomy_08genus', 'organism_taxonomy_09species']).size().reset_index(name='count')

# Find the most potent cases
most_potent = df.loc[df.groupby(['structure_nameTraditional'])['value'].idxmax()]

# Summary paragraph
summary_paragraph = f"In the dataset, {len(df)} compounds were identified across various taxa. The most potent compounds were determined based on their biological activity values, with {len(most_potent)} unique compounds exhibiting the highest activity within their respective families, genera, and species. These compounds include {', '.join(most_potent['structure_nameTraditional'].tolist())}."

# Recurrence of compounds in each taxon
recurrence_summary = compound_counts.groupby(['organism_taxonomy_06family', 'organism_taxonomy_08genus', 'organism_taxonomy_09species']).size().reset_index(name='compound_count')

# Write the summary of compound recurrence in each taxon to a CSV file
recurrence_summary.to_csv('compound_recurrence_summary.csv', index=False)


In [11]:
summary_paragraph

'In the dataset, 37 compounds were identified across various taxa. The most potent compounds were determined based on their biological activity values, with 14 unique compounds exhibiting the highest activity within their respective families, genera, and species. These compounds include (2S)-2-amino-5-(diaminomethylideneazaniumyl)pentanoate, (2S)-2-ammonio-4-(carbamimidamidooxy)butanoate, 2,5-Dihydroxybenzoic acid, 2-Carboxyphenolate;hydron, Chrysin, Friedelin, Gallic Acid, Melatonin, Oleanolic acid, Oleanonic acid, Prunetin, Quercetin, Taraxerol, beta-Amyrin acetate.'

In [12]:
recurrence_summary

,inchikey,molecule_chembl_id,target_pref_name,units,value,wikidata_id,structure_inchikey,structure_wikidata,structure_inchi,structure_molecular_formula,...,structure_taxonomy_npclassifier_02superclass,structure_taxonomy_npclassifier_03class,organism_wikidata,organism_taxonomy_ottid,organism_taxonomy_06family,organism_taxonomy_08genus,organism_taxonomy_09species,reference_wikidata,reference_doi,chemical_superclass
2,ODKSFYDXXFIFQN-BYPYZUCNSA-N,CHEMBL1485,"Nitric oxide synthase, inducible",%,50.00,http://www.wikidata.org/entity/Q173670,ODKSFYDXXFIFQN-BYPYZUCNSA-N,http://www.wikidata.org/entity/Q173670,InChI=1S/C6H14N4O2/c7-4(5(11)12)2-1-3-10-6(8)9...,C6H14N4O2,...,Small peptides,Aminoacids,http://www.wikidata.org/entity/Q11127145,1011944.0,Fabaceae,Indigofera,Indigofera hirsuta,http://www.wikidata.org/entity/Q39251683|http:...,10.1021/JF60189A002|10.1021/JF60189A002|10.102...,Amino acids and Peptides-Small peptides
1,FSBIGDSBMBYOPN-VKHMYHEASA-N,CHEMBL443732,"Nitric oxide synthase, inducible",uM,60.00,http://www.wikidata.org/entity/Q418408,FSBIGDSBMBYOPN-VKHMYHEASA-N,http://www.wikidata.org/entity/Q418408,InChI=1S/C5H12N4O3/c6-3(4(10)11)1-2-12-9-5(7)8...,C5H12N4O3,...,Small peptides,Aminoacids,http://www.wikidata.org/entity/Q11127145,1011944.0,Fabaceae,Indigofera,Indigofera hirsuta,http://www.wikidata.org/entity/Q39251683|http:...,10.1021/JF60189A002|10.1021/JF60189A002|10.102...,Amino acids and Peptides-Small peptides
21,WXTMDXOMEHJXQO-UHFFFAOYSA-N,CHEMBL1461,"Nitric oxide synthase, inducible",%,24.00,http://www.wikidata.org/entity/Q417831,WXTMDXOMEHJXQO-UHFFFAOYSA-N,http://www.wikidata.org/entity/Q417831,InChI=1S/C7H6O4/c8-4-1-2-6(9)5(3-4)7(10)11/h1-...,C7H6O4,...,Phenolic acids (C6-C1),Simple phenolic acids,http://www.wikidata.org/entity/Q2717303,869024.0,Fabaceae,Gliricidia,Gliricidia sepium,http://www.wikidata.org/entity/Q86783884,10.1007/BF00982301,Shikimates and Phenylpropanoids-Phenolic acids...
9,YGSDEFSMJLZEOE-UHFFFAOYSA-N,CHEMBL424,"Nitric oxide synthase, inducible",%,13.00,http://www.wikidata.org/entity/Q193572,YGSDEFSMJLZEOE-UHFFFAOYSA-N,http://www.wikidata.org/entity/Q193572,"InChI=1S/C7H6O3/c8-6-4-2-1-3-5(6)7(9)10/h1-4,8...",C7H6O3,...,Phenolic acids (C6-C1),Simple phenolic acids,http://www.wikidata.org/entity/Q161093,524094.0,Apocynaceae,Catharanthus,Catharanthus roseus,http://www.wikidata.org/entity/Q46104950|http:...,10.1016/J.PHYTOCHEM.2009.01.009|10.1016/J.PHYT...,Shikimates and Phenylpropanoids-Phenolic acids...
11,RTIXKCRFFJGDFG-UHFFFAOYSA-N,CHEMBL117,"Nitric oxide synthase, inducible",%,8.90,http://www.wikidata.org/entity/Q973741,RTIXKCRFFJGDFG-UHFFFAOYSA-N,http://www.wikidata.org/entity/Q973741,InChI=1S/C15H10O4/c16-10-6-11(17)15-12(18)8-13...,C15H10O4,...,Flavonoids,Flavones,http://www.wikidata.org/entity/Q1760533,197754.0,Combretaceae,Laguncularia,Laguncularia racemosa,http://www.wikidata.org/entity/Q39761461,10.1016/J.PHYTOCHEM.2009.11.008,Shikimates and Phenylpropanoids-Flavonoids
14,OFMXGFHWLZPCFL-SVRPQWSVSA-N,CHEMBL485998,"Nitric oxide synthase, inducible",uM,50.00,http://www.wikidata.org/entity/Q15410972,OFMXGFHWLZPCFL-SVRPQWSVSA-N,http://www.wikidata.org/entity/Q15410972,"InChI=1S/C30H50O/c1-20-21(31)9-10-22-27(20,5)1...",C30H50O,...,Triterpenoids,Friedelane triterpenoids,http://www.wikidata.org/entity/Q1949712,885278.0,Asteraceae,Ageratum,Ageratum conyzoides,http://www.wikidata.org/entity/Q105026194,10.1016/S0031-9422(00)97171-1,Terpenoids-Triterpenoids
4,LNTHITQWFMADLM-UHFFFAOYSA-N,CHEMBL288114,"Nitric oxide synthase, inducible",uM,50.00,http://www.wikidata.org/entity/Q375837,LNTHITQWFMADLM-UHFFFAOYSA-N,http://www.wikidata.org/entity/Q375837,InChI=1S/C7H6O5/c8-4-1-3(7(11)12)2-5(9)6(4)10/...,C7H6O5,...,Phenolic acids (C6-C1),Gallotannins,http://www.wikidata.org/entity/Q1428973,434080.0,Fabaceae,Caesalpinia,Caesalpinia pulcherrima,http://www.wikidata.org/entity/Q44112346|http:...,10.1021/NP0201523|10.1021/NP0201523,Shikimates and Phenylpropanoids-Phenolic acids...
10,DRLFMBDRBRZALE-UHFFFAOYSA-N,CHE